In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import calendar
import lightgbm as lgb

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
international_trade_csv = pd.read_csv('data/international_trade.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [3]:
dic_train = {}
for idx, time in enumerate(train_df['timestamp'].unique()):
    time = time.replace('-', '')
    dic_train[time] = idx+1
    
dic_test = {}
for idx, time in enumerate(test_df['timestamp'].unique()):
    time = time.replace('-', '')
    dic_test[time] = idx+1524

In [4]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train_df['year'] = train_df['timestamp'].apply(lambda x : int(x[0:4]))
train_df['month'] = train_df['timestamp'].apply(lambda x : int(x[5:7]))
train_df['day'] = train_df['timestamp'].apply(lambda x : int(x[8:10]))
train_df['weekday'] = train_df['timestamp'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').weekday())
train_df['prod_ID'] = train_df['ID'].apply(lambda x: x[0:6])
train_df['d'] = train_df['ID'].apply(lambda x: f'd_{dic_train[x[7:]]}')
# train_df.drop(['supply(kg)', 'timestamp'], axis=1, inplace=True)

test_df['year'] = test_df['timestamp'].apply(lambda x : int(x[0:4]))
test_df['month'] = test_df['timestamp'].apply(lambda x : int(x[5:7]))
test_df['day'] = test_df['timestamp'].apply(lambda x : int(x[8:10]))
test_df['weekday'] = test_df['timestamp'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d').weekday())
test_df['prod_ID'] = test_df['ID'].apply(lambda x: x[0:6])
test_df['d'] = test_df['ID'].apply(lambda x: f'd_{dic_test[x[7:]]}')
# test_df.drop(['timestamp'], axis=1, inplace=True)

이동 평균

In [6]:
train_df.rename(columns={'price(원/kg)':'price', 'supply(kg)': 'supply'}, inplace=True)

In [7]:
def get_moving_average(df):
    for win in [1, 2, 4, 8]:
        df['rm_diff_price_{}'.format(win)] = df[['item', 'corporation', 'location', 'price']].groupby(
            ['item', 'corporation', 'location'])['price'].transform(lambda x : x.rolling(win).mean())
        df['rm_diff_price_{}'.format(win)] = ((df['price'] - df['rm_diff_price_{}'.format(win)]
                                                  )/df['price']).round(3)
    return df
train_df = get_moving_average(train_df)

In [8]:
def lags_wins(df):
    lags = [7, 14, 28]
    lag_cols = [f"lag_{lag}" for lag in lags]
    for lag, lag_col in zip(lags, lag_cols):
        df[lag_col] = df[["prod_ID","price"]].groupby("prod_ID")["price"].shift(lag)

    wins = [7, 14, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            df[f"rmean_{lag}_{win}"] = df[["prod_ID", lag_col]].groupby("prod_ID")[lag_col].transform(lambda x : x.rolling(win).mean())
    return df
train_df = lags_wins(train_df)

In [9]:
train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,day,...,lag_28,rmean_7_7,rmean_14_7,rmean_28_7,rmean_7_14,rmean_14_14,rmean_28_14,rmean_7_28,rmean_14_28,rmean_28_28
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27,...,604.0,528.142857,372.428571,402.000000,450.285714,433.642857,355.142857,449.357143,394.392857,377.642857
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28,...,719.0,519.285714,390.428571,504.714286,454.857143,420.642857,377.285714,466.321429,398.964286,383.214286
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1,...,646.0,497.857143,416.714286,507.428571,457.285714,416.642857,393.571429,459.642857,405.107143,386.607143
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2,...,620.0,456.142857,464.714286,500.714286,460.428571,421.500000,406.785714,449.964286,414.142857,391.642857


In [10]:
valid_df = train_df.copy()
for day in range(1496, 1524):
    valid_df.loc[valid_df['d']==f'd_{day}', ['price', 'rm_diff_price_1', 'rm_diff_price_2', 'rm_diff_price_4',
       'rm_diff_price_8', 'lag_7', 'lag_14', 'lag_28', 'rmean_7_7',
       'rmean_14_7', 'rmean_28_7', 'rmean_7_14', 'rmean_14_14', 'rmean_28_14',
       'rmean_7_28', 'rmean_14_28', 'rmean_28_28']] = 0

In [11]:
valid_df

,ID,timestamp,item,corporation,location,supply,price,year,month,day,...,lag_28,rmean_7_7,rmean_14_7,rmean_28_7,rmean_7_14,rmean_14_14,rmean_28_14,rmean_7_28,rmean_14_28,rmean_28_28
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,0.0,2023,2,27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,0.0,2023,2,28,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,0.0,2023,3,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,0.0,2023,3,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
test_df['price'] = 0
test_df[['rm_diff_price_1', 'rm_diff_price_2', 'rm_diff_price_4',
       'rm_diff_price_8', 'lag_7', 'lag_14', 'lag_28', 'rmean_7_7',
       'rmean_14_7', 'rmean_28_7', 'rmean_7_14', 'rmean_14_14', 'rmean_28_14',
       'rmean_7_28', 'rmean_14_28', 'rmean_28_28']] = 0.0
test_df = pd.concat([train_df, test_df])

In [13]:
test_df

,ID,timestamp,item,corporation,location,supply,price,year,month,day,...,lag_28,rmean_7_7,rmean_14_7,rmean_28_7,rmean_7_14,rmean_14_14,rmean_28_14,rmean_7_28,rmean_14_28,rmean_28_28
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,NaN,0.0,2023,3,27,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1088,RD_F_J_20230328,2023-03-28,RD,F,J,NaN,0.0,2023,3,28,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1089,RD_F_J_20230329,2023-03-29,RD,F,J,NaN,0.0,2023,3,29,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1090,RD_F_J_20230330,2023-03-30,RD,F,J,NaN,0.0,2023,3,30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
category_cols = ['item','corporation', 'location', 'year', 'month', 'day', 'weekday']
useless_cols = ["ID", "prod_ID", "d", "supply"]
train_cols = train_df.columns[~train_df.columns.isin(useless_cols)]
df = train_df.copy()
days_train = ['d_'+str(c) for c in range(1, 1496)]
days_val = ['d_'+str(c) for c in range(1496, 1524)]
df = df.replace([np.inf, -np.inf], 0)
#df.iloc[:, -9:] = df.iloc[:, -9:].fillna(0.0)
X_train = df[df['d'].isin(days_train)==True][train_cols]
y_train = df[df['d'].isin(days_train)==True]["price"]
X_val_df = df[df['d'].isin(days_val)==True]
X_val_df["timestamp"] = pd.to_datetime(X_val_df["timestamp"])
X_val_dff = pd.DataFrame()
for delta in range(0, 28):
    day = datetime(2023, 2, 4) + timedelta(days=delta)
    vl = X_val_df.loc[X_val_df.timestamp == day]
    X_val_dff = pd.concat([X_val_dff, vl])

X_val = X_val_dff[train_cols]
y_val = X_val_dff["price"]

C:\Users\cji01\AppData\Local\Temp/ipykernel_6980/4239290619.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val_df["timestamp"] = pd.to_datetime(X_val_df["timestamp"])


In [ ]:
sample_submission

In [27]:
1523-28

1495

In [29]:
dic_train['20230303']

1523

In [30]:
dic_train['20230204']

1496

In [32]:
X_val

,timestamp,item,corporation,location,price,year,month,day,weekday,rm_diff_price_1,...,lag_28,rmean_7_7,rmean_14_7,rmean_28_7,rmean_7_14,rmean_14_14,rmean_28_14,rmean_7_28,rmean_14_28,rmean_28_28
1495,2023-02-04,TG,A,J,1864.0,2023,2,4,5,0.0,...,1834.0,1123.285714,1739.285714,1489.000000,1431.285714,1763.857143,1643.428571,1535.000000,1703.642857,1525.178571
3018,2023-02-04,TG,A,S,2656.0,2023,2,4,5,0.0,...,2475.0,1658.428571,2103.714286,1765.142857,1881.071429,2107.714286,1992.000000,1909.750000,2049.857143,1925.535714
4541,2023-02-04,TG,B,J,1839.0,2023,2,4,5,0.0,...,1484.0,1214.571429,1253.714286,1143.428571,1234.142857,1496.428571,1433.285714,1337.714286,1464.857143,1433.107143
6064,2023-02-04,TG,B,S,1884.0,2023,2,4,5,0.0,...,1918.0,1835.571429,2085.142857,1547.428571,1960.357143,2218.357143,1811.000000,1954.928571,2014.678571,1743.607143
7587,2023-02-04,TG,C,J,2542.0,2023,2,4,5,0.0,...,2074.0,1427.571429,1828.714286,1449.428571,1628.142857,1679.785714,1707.785714,1559.142857,1693.785714,1758.892857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53304,2023-03-03,BC,D,J,3059.0,2023,3,3,4,0.0,...,2144.0,1765.000000,1640.428571,1757.857143,1702.714286,1810.785714,1548.214286,1786.107143,1679.500000,1731.035714
54827,2023-03-03,BC,E,J,3045.0,2023,3,3,4,0.0,...,2469.0,1834.857143,1813.000000,1829.000000,1823.928571,1924.785714,1383.000000,1878.357143,1653.892857,1681.285714
56350,2023-03-03,BC,E,S,2939.0,2023,3,3,4,0.0,...,2691.0,2002.714286,1908.714286,2036.714286,1955.714286,2034.785714,1233.428571,2027.250000,1634.107143,1674.500000
57873,2023-03-03,CB,F,J,643.0,2023,3,3,4,0.0,...,539.0,593.714286,557.285714,582.000000,575.500000,514.071429,504.000000,550.964286,509.035714,471.000000


In [ ]:
def lgb_objective(trial: Trial) -> float:
    params_lgb = {
        
#         "learning_rate": trial.suggest_uniform("learning_rate", 0.03, 0.05),
        #"lambda_l1": trial.suggest_uniform("lambda_l1", 0.0, 1),
        #"lambda_l2": trial.suggest_uniform("lambda_l2", 0.0, 1),
        
        'boosting_type': 'gbdt',
#         'objective': 'tweedie',
        'metric': 'rmse',
        'verbose': -1,
        'boost_from_average': False,
        'n_estimators': 5000,
        "learning_rate": trial.suggest_uniform("learning_rate", 0.01, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 1500, 2500), 
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 3000, 5000),
        "max_bin": trial.suggest_int("max_bin", 200, 400),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.3, 0.5),
    }
    
    train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=category_cols)
    valid_data = lgb.Dataset(X_val, label = y_val, categorical_feature=category_cols)
                                                                                            # 진행상황 보고싶을때 None을 100으로
    lgbmodel = lgb.train(params_lgb, train_data, valid_sets=[valid_data], early_stopping_rounds=200,
                         feval=wrmsse_lgb, verbose_eval=None)
    
    return wrmsse